## 무신사 크롤링

In [26]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By # 특정 선택자로 값을 찾아옴.
from selenium.webdriver.common.keys import Keys #명령어 넣을때
from webdriver_manager.chrome import ChromeDriverManager
import time

service = Service(ChromeDriverManager().install())
options = Options()
#options.add_argument('--headless') #headless를 쓰지 않으면 Grapic 소스가 불필요하게 충돌할 수 있음
#options.add_argument('--disable-gpu') #셀레니움 사용시, headless를 사용하지 않으면 브라우저가 켜짐. Gpu 실행. 이런 비효율성을 상대 페이지에서는 감지가 가능함. headless 사용시 사용하면 좋음
options.add_argument('--disable-dev-shm-usage') #로컬컴퓨터 메모리쓰기
options.add_argument('--window-size=1920x1080')
options.add_argument('--start-maximized') #반응형 페이지 방어
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36")
options.add_argument('--lang=ko_KR')

driver = webdriver.Chrome(service=service, options=options) #드라이버 실행

driver.get("https://www.musinsa.com/main/musinsa/recommend?gf=A")
time.sleep(3)

#검색버튼 누르게하기
search_button = driver.find_element(By.CLASS_NAME,"sc-dzagxs-2")
search_button.click()
time.sleep(2)

# 반팔버튼 누르기 - 순위가 
popular_keywords = driver.find_elements(By.CSS_SELECTOR,".search-home-popular-wrap .sc-1xzm8bl-1")
target_keyword = popular_keywords[2].text
print("목표 키워드 :", target_keyword)

#반팔 텍스트 검색어에 넣기
search_input = driver.find_element(By.CLASS_NAME,"search-home-search-bar-keyword")
search_input.clear()
search_input.send_keys(target_keyword) # 이 코드로 텍스트도 칠 수 있음!
time.sleep(1)
#엔터키를 자동으로 치게하기
search_input.send_keys(Keys.RETURN)
time.sleep(5)

print("검색완료. 현재 페이지 타이틀 : ", driver.title)

time.sleep(2)
driver.quit()

목표 키워드 : 반팔
검색완료. 현재 페이지 타이틀 :  반팔 | 무신사 추천 상품


## 무신사 크롤링 2 

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By # 특정 선택자로 값을 찾아옴.
from selenium.webdriver.common.keys import Keys #명령어 넣을때
from webdriver_manager.chrome import ChromeDriverManager
import time
import re # re : 어떤 값을 제거하는 remove 객체!

service = Service(ChromeDriverManager().install())
options = Options()
#options.add_argument('--headless') #headless를 쓰지 않으면 Grapic 소스가 불필요하게 충돌할 수 있음
#options.add_argument('--disable-gpu') #셀레니움 사용시, headless를 사용하지 않으면 브라우저가 켜짐. Gpu 실행. 이런 비효율성을 상대 페이지에서는 감지가 가능함. headless 사용시 사용하면 좋음
options.add_argument('--disable-dev-shm-usage') #로컬컴퓨터 메모리쓰기
options.add_argument('--window-size=1920x1080')
options.add_argument('--start-maximized') #반응형 페이지 방어
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36")
options.add_argument('--lang=ko_KR')

driver = webdriver.Chrome(service=service, options=options) #드라이버 실행

driver.get("https://www.musinsa.com/main/musinsa/recommend?gf=A")
time.sleep(3)

#검색버튼 누르게하기
search_button = driver.find_element(By.CLASS_NAME,"sc-dzagxs-2")
search_button.click()
time.sleep(2)

# 반팔버튼 누르기 - 순위가 
popular_keywords = driver.find_elements(By.CSS_SELECTOR,".search-home-popular-wrap .sc-1xzm8bl-1")
target_keyword = popular_keywords[1].text # 인기키워드 몇번째인지
print("목표 키워드 :", target_keyword)

#반팔 텍스트 검색어에 넣기
search_input = driver.find_element(By.CLASS_NAME,"search-home-search-bar-keyword")
search_input.clear()
search_input.send_keys(target_keyword)
time.sleep(1)
#엔터키를 자동으로 치게하기
search_input.send_keys(Keys.RETURN)
time.sleep(5)

print("검색완료. 현재 페이지 타이틀 : ", driver.title)

scroll_pause = 2
#10번반복하는 동안
for _ in range (10):
    #스크롤 밑으로 내림
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause) #반복할때마다 2초 쉼

products = driver.find_elements(By.CSS_SELECTOR,"div.sc-d36st-1.elXjMR")[:100] #계속찾아오는거 방지. 100개만 찾아와
product_data = []

for product in products:
    try :
        name = product.find_element(By.CSS_SELECTOR,"div.sc-cMuefe.jIlLeV a.gtm-select-item span").text.strip() #후손 선택자 : 한칸 띄우기
        url = product.find_element(By.CSS_SELECTOR,"div.sc-cMuefe.jIlLeV a.gtm-select-item").get_attribute("href") #속성값에 a 값인 url 넣어서 가져오기
        review_info = product.find_elements(By.CSS_SELECTOR,"span.text-etc_11px_reg.text-yellow.font-pretendard")[1].text.strip() #리뷰 평점 찾기
        review_count = int(re.sub(r"[^\d]","",review_info)) if review_info else 0  # review_info에 리뷰 값이 있으면 값을 주고 없으면 0을 준다.
        # r = 정규표현식을 쓰겠다. ^\d ^반대 \d 는 숫자! 숫자가 아닌 것을 ""(빈값)으로 sub(대체해라)
        product_data.append({"상품명": name, "URL": url, "리뷰수" : review_count})
    except :
        continue
        
print(product_data)

driver.quit()

목표 키워드 : 반팔
검색완료. 현재 페이지 타이틀 :  반팔 | 무신사 추천 상품
[{'상품명': 'Austin 반팔 티셔츠', 'URL': 'https://www.musinsa.com/products/5123733', '리뷰수': 23}, {'상품명': '오버 핏 스트라이프 티셔츠_블루', 'URL': 'https://www.musinsa.com/products/4168334', '리뷰수': 4}, {'상품명': '리본 셔링 오프숄더 티셔츠 백멜란지', 'URL': 'https://www.musinsa.com/products/4903054', '리뷰수': 43}, {'상품명': 'BRUSH FADING HALF SLEEVE T-SHIRTS (BLACK)', 'URL': 'https://www.musinsa.com/products/3249016', '리뷰수': 4}, {'상품명': '먼데이루틴 브로콜리 반팔티셔츠_화이트', 'URL': 'https://www.musinsa.com/products/2463159', '리뷰수': 656}, {'상품명': 'ROCK STAR APPLIQUE TEE WHITE(MG2FMMT546A)', 'URL': 'https://www.musinsa.com/products/5161178', '리뷰수': 2}, {'상품명': '어드벤쳐 폰트 피그먼트 오버핏 반팔티셔츠 다크그레이 FST705', 'URL': 'https://www.musinsa.com/products/2004196', '리뷰수': 2}, {'상품명': '[반팔ver]나그랑뒷반팔티셔츠(PBE03S-WHTW004)', 'URL': 'https://www.musinsa.com/products/4943233', '리뷰수': 83}, {'상품명': '짱구 액션가면 반팔 야광티셔츠', 'URL': 'https://www.musinsa.com/products/4112489', '리뷰수': 40}, {'상품명': '우먼즈 B.LIFE 오버핏 반팔티', 'URL': 'https://

## 판다스 쓰기

In [17]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By # 특정 선택자로 값을 찾아옴.
from selenium.webdriver.common.keys import Keys #명령어 넣을때
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd #판다스 가져오고 pd라는 이름으로 쓴다
import time
import re # re : 어떤 값을 제거하는 remove 객체!

service = Service(ChromeDriverManager().install())
options = Options()
#options.add_argument('--headless') #headless를 쓰지 않으면 Grapic 소스가 불필요하게 충돌할 수 있음
#options.add_argument('--disable-gpu') #셀레니움 사용시, headless를 사용하지 않으면 브라우저가 켜짐. Gpu 실행. 이런 비효율성을 상대 페이지에서는 감지가 가능함. headless 사용시 사용하면 좋음
options.add_argument('--disable-dev-shm-usage') #로컬컴퓨터 메모리쓰기
options.add_argument('--window-size=1920x1080')
options.add_argument('--start-maximized') #반응형 페이지 방어
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36")
options.add_argument('--lang=ko_KR')

driver = webdriver.Chrome(service=service, options=options) #드라이버 실행

driver.get("https://www.musinsa.com/main/musinsa/recommend?gf=A")
time.sleep(3)

#검색버튼 누르게하기
search_button = driver.find_element(By.CLASS_NAME,"sc-dzagxs-2")
search_button.click()
time.sleep(2)

# 반팔버튼 누르기 - 순위가 
popular_keywords = driver.find_elements(By.CSS_SELECTOR,".search-home-popular-wrap .sc-1xzm8bl-1")
target_keyword = popular_keywords[2].text
print("목표 키워드 :", target_keyword)

#반팔 텍스트 검색어에 넣기
search_input = driver.find_element(By.CLASS_NAME,"search-home-search-bar-keyword")
search_input.clear()
search_input.send_keys(target_keyword)
time.sleep(1)
#엔터키를 자동으로 치게하기
search_input.send_keys(Keys.RETURN)
time.sleep(5)

print("검색완료. 현재 페이지 타이틀 : ", driver.title)

scroll_pause = 2
#10번반복하는 동안
for _ in range (10):
    #스크롤 밑으로 내림
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause) #반복할때마다 2초 쉼

products = driver.find_elements(By.CSS_SELECTOR,"div.sc-d36st-1.elXjMR")[:100] #계속찾아오는거 방지. 100개만 찾아와
product_data = []

for product in products:
    try :
        name = product.find_element(By.CSS_SELECTOR,"div.sc-cMuefe.jIlLeV a.gtm-select-item span").text.strip() #후손 선택자 : 한칸 띄우기
        url = product.find_element(By.CSS_SELECTOR,"div.sc-cMuefe.jIlLeV a.gtm-select-item").get_attribute("href") #속성값에 a 값인 url 넣어서 가져오기
        review_info = product.find_elements(By.CSS_SELECTOR,"span.text-etc_11px_reg.text-yellow.font-pretendard")[1].text.strip() #리뷰 평점 찾기
        review_count = int(re.sub(r"[^\d]","",review_info)) if review_info else 0  # review_info에 리뷰 값이 있으면 값을 주고 없으면 0을 준다.
        # r = 정규표현식을 쓰겠다. ^\d ^반대 \d 는 숫자! 숫자가 아닌 것을 ""(빈값)으로 sub(대체해라)
        product_data.append({"상품명": name, "URL": url, "리뷰수" : review_count})
    except :
        continue
# 판다스!!!!!!!!###############################################
df = pd.DataFrame(product_data)
df_top30 = df.sort_values(by="리뷰수", ascending=False).head(30).reset_index(drop=True) 
#리뷰수를 기준으로 sorting을 시킴, ascending 시키지마라 = 내림차순, 30개 잘라라, 나머지는 버려라

driver.quit()
df_top30.head()

목표 키워드 : 반팔
검색완료. 현재 페이지 타이틀 :  반팔 | 무신사 추천 상품


,상품명,URL,리뷰수
0,DOTTY RIBBON SHIRRING T-SHIRT_RED,https://www.musinsa.com/products/5082524,22
1,OUTER HEART TEE (BURGUNDY),https://www.musinsa.com/products/4249776,7
2,로고 라인업 피그먼트 반팔티 크림,https://www.musinsa.com/products/3182326,3
3,[170g] 빅 로고 반팔 티 (JR5TSU091WH),https://www.musinsa.com/products/1090694,2


## 진짜 리뷰(텍스트)들을 추출하는 방법 

In [ ]:
from openpyxl import Workbook

In [18]:
from openpyxl import Workbook
######################리뷰 텍스트들 추출 ###################################
# 1, 위에 openpyxl!
driver = webdriver.Chrome(service=service, options=options)
all_reviews = []

for idx, row in df_top30.iterrows(): #iterrows : 1개의 행을 가져오고 반복할때마다 1개의 값을 찾아오게 하는 함수
    print(f"{idx + 1}번째 상품리뷰 수집 중 :", {row['상품명']})
    driver.get(row["URL"]) #row라는 행 안에 URL이라는 값에 매칭되는 애를 찾아오겠다
    time.sleep(2)

#오류 발생할 수도 있으니까 try
    try :
        review_tab = driver.find_element(By.XPATH,"//button[@data-button-id='prd_review_tab']") # id 안전빵으로 찾아오고 xpath
        review_tab.click()
        time.sleep(2)
        #모든리뷰 X 한 상품당 10개정도만 스크롤해서 찾아오기
        for _ in range(3) :
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") 
            time.sleep(1.5)

        # 스크롤할때마다 리뷰 찾아오기
        review_elements = driver.find_elements(By.CSS_SELECTOR,"span.text-body_13px_reg.text-black.font-pretendard")
        for review in review_elements[:10] :
            text = review.text.strip()
            if text : 
                all_reviews.append({
                    "상품명" : row["상품명"], #row상품명에서 가져옴
                    "리뷰" : text
                }) # 비어있는 리스트에 값추가

    except Exception as e:
        print("리뷰 수집 실패 : ", e)
        continue

driver.quit()
#엑셀 저장
wb = Workbook()
ws = wb.active
ws.title = "무신사 리뷰"

ws. append(["상품명","리뷰"])

#각각의 리뷰안에 상품명 한쌍으로 만들기
for r in all_reviews:
    ws.append([r["상품명"],r["리뷰"]])

wb.save("musinsa_top30_reviews.xlsx")
print("⭐엑셀 저장완료 : musinsa_top30_reviews.xlsx")

1번째 상품리뷰 수집 중 : {'DOTTY RIBBON SHIRRING T-SHIRT_RED'}
2번째 상품리뷰 수집 중 : {'OUTER HEART TEE (BURGUNDY)'}
3번째 상품리뷰 수집 중 : {'로고 라인업 피그먼트 반팔티 크림'}
4번째 상품리뷰 수집 중 : {'[170g] 빅 로고 반팔 티 (JR5TSU091WH)'}
⭐엑셀 저장완료 : musinsa_top30_reviews.xlsx


In [43]:
pip install sqlalchemy

In [11]:
import pymysql
import pandas as pd
from sqlalchemy import create_engine # sql테이블로 바로 보냄

EXCEL_PATH = "musinsa_top30_reviews_0811.xlsx"
DB_NAME = "musinsa_db_v2"
TABLE_NAME = "reviews"
USER = "root"
PASSWORD = "asdf1122"
HOST = "127.0.0.1"
PORT = 3306
df = pd.read_excel(EXCEL_PATH)

conn = pymysql.connect(host=HOST, user = USER, password = PASSWORD, port = PORT)
cursor = conn.cursor()

cursor.execute(
f"CREATE DATABASE IF NOT EXISTS {DB_NAME} CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;"
)
conn.close()

engine_str = f"mysql+pymysql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB_NAME}?charset=utf8mb4"
engine = create_engine(engine_str)

df.to_sql(name=TABLE_NAME, con=engine, if_exists="replace", index=False)

print(f"⭐데이터가 MYSQL[{DB_NAME}]의 테이블 [{TABLE_NAME}]에 성공적으로 저장되었습니다.")

⭐데이터가 MYSQL[musinsa_db_v2]의 테이블 [reviews]에 성공적으로 저장되었습니다.


In [ ]:
# 자동화를 위해 구글 스프레드 시트로 보내야함. LIBRARY 설치! -> gspread

In [12]:
pip install gspread


   ---------------------------------------- 0/6 [rsa]
   ---------------------------------------- 0/6 [rsa]
   ------ --------------------------------- 1/6 [oauthlib]
   ------ --------------------------------- 1/6 [oauthlib]
   ------ --------------------------------- 1/6 [oauthlib]
   ------ --------------------------------- 1/6 [oauthlib]
   ------ --------------------------------- 1/6 [oauthlib]
   ------------- -------------------------- 2/6 [requests-oauthlib]
   -------------------- ------------------- 3/6 [google-auth]
   -------------------- ------------------- 3/6 [google-auth]
   -------------------- ------------------- 3/6 [google-auth]
   -------------------- ------------------- 3/6 [google-auth]
   -------------------- ------------------- 3/6 [google-auth]
   -------------------------- ------------- 4/6 [google-auth-oauthlib]
   --------------------------------- ------ 5/6 [gspread]
   --------------------------------- ------ 5/6 [gspread]
   ----------------------------

In [ ]:
# 구글은 외부에서 컨트을 하려고 할 떄, 우리의 존재를 인증하면 구글이 접근을 허용해줌! 인증을 도와주는 LIBRARY -> oauth2client

In [14]:
pip install oauth2client


   -------------------- ------------------- 1/2 [oauth2client]
   -------------------- ------------------- 1/2 [oauth2client]
   ---------------------------------------- 2/2 [oauth2client]

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# gspread 로 데이터를 보낼 때 형태소 분석을 해달라고 할 것! keyword추츨을 위해!

In [21]:
from konlpy.tag import Okt
from oauth2client.service_account import ServiceAccountCredentials # 구글이 운영하는 API를 가져와서 검증된 사용자임을 인증!
import gspread # 구글 스프레드 시트에 접근할 수 있도록!
import pandas as pd

# dataframe요소에 엑셀을 판다스에 넣기!
df = pd.read_excel("musinsa_top30_reviews_0811.xlsx")
#형태소 분석하기 함수활용
okt = Okt() 
# 리뷰에 있는 값만 찾아오기
# "리뷰"라는 필드명의 컬럼 값을 찾아오게 할 수 있음! 'apply' -> 적용하다!
df["리뷰_키워드"] = df["리뷰"].apply(lambda x: ", ".join(okt.nouns(str(x)))) 
#                ↳ lambda x : 리뷰 컬럼의 각 행(row)을 하나씩 함수에 전달, 'join' : 이 리스트를 쉼표로 구분된 하나의 문자열로 변환
#                ↳ okt.nouns()는 텍스트에서 명사만 추출하여 리스트로 반환, 'str': 숫자로 들어온 값도 문자로 변환
#                ↳ 쪼개진 명사를 ,를 중심으로 구분지어서 하나의 문자열로 만들어줌.
#                ↳ 다음 열에 리뷰_키워드로 각 값들을 하나씩 넣음!!!

######구글 스프레드 시트를 사용하기 위해 구글에 인증 받기!

scope = ["https://spreadsheets.google.com/feeds","https://www.googleapis.com/auth/drive"]
# scope는 구글 API에 접근할 수 있는 권한 범위를 정의
# 스프레드시트와 드라이브 접근 권한을 요청하는 것이며, URL이 아닌 권한 범위

creds = ServiceAccountCredentials.from_json_keyfile_name("fine-arbor-468703-s6-70517b51b42c.json",scope) 
# 인증절차를 받아오는 결과 값을 받아오기.구글 클라우드 콘솔에서 생성한 서비스 계정 키, 
# scope와 함께 사용하여 구글 API에 접근 권한을 얻음

client = gspread.authorize(creds)
# 인증을 받아오는 역할 인증받은 'creds'를 통해 구글 스프레드 시트의 인증을 받음!

# client가 가지고 있는 "무신사 리뷰 분석_v1" 스프레드시트를 열기
spreadsheet = client.open("무신사 리뷰 분석_v1")
# spreadsheet의 첫 번째 시트를 가져오는 속성 = spreadsheet.worksheet("시트이름")
sheet1 = spreadsheet.sheet1
# sheet1를 초기화시킴
sheet1.clear()
#sheet1에 값을 추가한다! = 두 리스트를 합쳐서 헤더와 데이터를 함께 시트에 update! 
#각각의 데이터프레임의 columns 이름을 리스트로 변환 
# + 데이터프레임의 모든 값을 리스트의 리스트 형태로 변환
sheet1.update([df.columns.values.tolist()] + df.values.tolist())


# try-except 블록은 "키워드 분석" 시트가 이미 존재하는 경우 에러를 방지
try:
    # 기존의 것은 놔두고 스프레드 시트에 새로운 값을 추가하겠다. 제목은 "키워드 분석" 행은 1000개 열은 20개
    spreadsheet.add_worksheet(title="키워드 분석", rows="1000",cols="20")
except :
    # 없으면 패스!
    pass

#시트2번 생성!
sheet2 = spreadsheet.worksheet("키워드 분석")
sheet2.clear()
#sheet2에 값추가!
sheet2.update([["상품명", "리뷰", "리뷰_키워드"]] + df[["상품명","리뷰", "리뷰_키워드"]].values.tolist())
# df[["상품명","리뷰", "리뷰_키워드"]]는 데이터프레임에서 특정 열만 선택하는 코드
# 데이터프레임에서 values.tolist()를 통해 데이터 값만 리스트로 변환
# 첫 번째 리스트 [["상품명", "리뷰", "리뷰_키워드"]]는 헤더(열 이름)를 지정하고, 그 다음에 실제 데이터 값들이 추가

print("⭐구글 스프레드시트의 원본 + 키워드 분석자료 업데이트 완료")

⭐구글 스프레드시트의 원본 + 키워드 분석자료 업데이트 완료
